# Effects on Information transmission of STP
## Rate-based STP model
We investigate the information transmission of STP by studying the time average response of short-term plasticity. 
By doing so, the spiking-based STP model is transfered to rate-based STP model, which can be written as,
$$\frac{du(t)}{dt}=-\frac{u(t)}{\tau_f}+U[1-u(t)]R(t)$$
$$\frac{dx(t)}{dt}=\frac{1-x(t)}{\tau_d}-u^+x(t)R(t)$$
$$g(t) = \tau_s g_{max} u^+ x(t)R(t)$$
$$u^+ = u+U(1-u)$$
### Customize the rate-based STP model

In [ ]:
class STP_rate(bp.DynamicalSystemNS): 
    def __init__(self, tau_d=500., tau_f=500., U=0.15, tau=8., g_max=0.1, method='exp_auto'):
        super(STP_rate, self).__init__()
        # 初始化参数
        self.tau_d = tau_d
        self.tau_f = tau_f
        self.tau = tau
        self.g_max = g_max
        self.U = U
        # 初始化变量
        self.x = bm.Variable(bm.ones(1))
        self.u = bm.Variable(bm.zeros(1))
        self.g = bm.Variable(bm.zeros(1))

        # 定义积分函数
        self.integral = bp.odeint(method=method, f=self.derivative)
    @property
    def derivative(self):
        du = lambda u, t, input: -u / self.tau_f + self.U * (1 - u) * input #input指的是pre-synaptic firing rate R(t)
        dx = lambda x, t, u, input: (1 - x) / self.tau_d - (u + self.U * (1 - u)) * x * input
        return bp.JointEq([du, dx])

    def update(self, input_):
        # 更新各个变量
        u, x = self.integral(self.u, self.x, bp.share['t'],  input_, bm.dt)
        self.u.value = u
        self.x.value = x
        self.g.value = self.tau * self.g_max * (u + self.U * (1 - u)) * x * input_

### Compare the simulation results of spiking-based STP model and rate-based STP model
To verify whether the rate-based model replicate the time-average response spiking-based STP model, we compare the simulation results of spiking-based STP and rate-based STP,

In [1]:

def compare_spike_rate_Stp(tau_d=500., tau_f=10., U=0.15):
    T1 = 500
    T2 = 500
    T3 = 500
    fr_1 = 15
    fr_2 = 30
    fr_3 = 80
    # 假设pre-synaptic spike 的频率在0-1000ms是1hz，在1000-2000ms是10hz，在2000-3000ms是20hz，
    # 模拟spike based stp,
    spike_1 = Pre_spike_generator(type='Regular', fr=fr_1, T=T1)
    spike_2 = Pre_spike_generator(type='Regular', fr=fr_2, T=T2)
    spike_3 = Pre_spike_generator(type='Regular', fr=fr_3, T=T3)
    spike_train = np.concatenate((spike_1, spike_2, spike_3))
    spike_time = np.where(spike_train == 1)[0]
    neu1 = bp.neurons.SpikeTimeGroup(1, times=spike_time, indices=[0] * len(spike_time))
    neu2 = bp.neurons.LIF(1)
    syn1 = STP(neu1, neu2, conn=bp.connect.All2All(), tau_d=tau_d, tau_f=tau_f, U=U)
    net = bp.Network(pre=neu1, syn=syn1, post=neu2)
    runner_spike = bp.DSRunner(net, monitors=['pre.spike', 'post.V', 'syn.g', 'syn.x', 'syn.u', 'post.input'])
    runner_spike.run(T1 + T2 + T3)
    
    # 模拟rate based stp,
    dyn_sys = STP_rate(tau_d=tau_d, tau_f=tau_f, U=U)
    # 分段电流
    inputs = bp.inputs.section_input(values=[fr_1 * 1e-3, fr_2 * 1e-3, fr_3 * 1e-3],
                                     durations=[T1, T2, T3])
    # 运行模拟
    runner_rate = bp.DSRunner(dyn_sys, monitors=['u', 'x', 'g'])
    runner_rate.run(inputs=inputs)

    # 可视化
    fig1, ax = plt.subplots(4, 1, figsize=(6, 6))
    ax[0].plot(runner_spike.mon.ts, runner_spike.mon['pre.spike'][:, 0], label='pre.spike')
    ax[0].legend()
    ax[0].set_xlabel(r'$t$ (ms)')
    ax[0].set_ylabel('spike')
    ax[0].spines['right'].set_color('none')
    ax[0].spines['top'].set_color('none')
    x_spike = runner_spike.mon['syn.x'][:, 0]
    x_rate = runner_rate.mon['x'][:, 0]
    ax[1].plot(runner_spike.mon.ts, x_spike, label='spike-based model')
    ax[1].plot(runner_rate.mon.ts, x_rate, label='rate-based model')
    ax[1].legend()
    ax[1].set_xlabel(r'$t$ (ms)')
    ax[1].set_ylabel('x')
    ax[1].spines['right'].set_color('none')
    ax[1].spines['top'].set_color('none')
    u_spike = runner_spike.mon['syn.u'][:, 0]
    u_rate = runner_rate.mon['u'][:, 0]
    ax[2].plot(runner_spike.mon.ts, u_spike, label='spike-based model')
    ax[2].plot(runner_rate.mon.ts, u_rate, label='rate-based model')
    ax[2].legend()
    ax[2].set_xlabel(r'$t$ (ms)')
    ax[2].set_ylabel('u')
    ax[2].spines['right'].set_color('none')
    ax[2].spines['top'].set_color('none')

    g_spike = runner_spike.mon['syn.g'][:, 0]
    g_rate = runner_rate.mon['g'][:, 0]
    ax[3].plot(runner_spike.mon.ts, g_spike, label='spike-based model')
    ax[3].plot(runner_rate.mon.ts, g_rate, label='rate-based model')
    ax[3].legend()
    ax[3].set_xlabel(r'$t$ (ms)')
    ax[3].set_ylabel('g')
    ax[3].spines['right'].set_color('none')
    ax[3].spines['top'].set_color('none')
    plt.show()
compare_spike_rate_Stp()

NameError: name 'Pre_spike_generator' is not defined